# Data Preprocessing

In [118]:
# Constants
DATA_PATH   = "../Data/Raw/survey_results_public2022.csv"
EXPORT_PATH = "../Data/Processed/1_preprocessed_df.pkl"

ROLE_COL = ['DevType']
CORE_COLS = [
             'VersionControlSystem',  
             'LanguageHaveWorkedWith',  'LanguageWantToWorkWith',  
             'DatabaseHaveWorkedWith',  'DatabaseWantToWorkWith',  
             'PlatformHaveWorkedWith',  'PlatformWantToWorkWith',  
             'WebframeHaveWorkedWith',  'WebframeWantToWorkWith',  
             'MiscTechHaveWorkedWith','MiscTechWantToWorkWith',  
             'ToolsTechHaveWorkedWith','ToolsTechWantToWorkWith',                                    
             'NEWCollabToolsHaveWorkedWith',  'NEWCollabToolsWantToWorkWith'
]

USEFUL_COLS= ['Employment','RemoteWork',
              'MainBranch','CodingActivities','ProfessionalTech',
              'LearnCode', 'LearnCodeOnline', 'LearnCodeCoursesCert',
              'WorkExp', 'YearsCode', 'YearsCodePro', 'EdLevel',
              'OrgSize', 'Country',
              'ConvertedCompYearly', 'Currency', 'CompTotal', 'CompFreq']

In [119]:
# Load packages
import pandas as pd 
import numpy as np
import logging
import pickle
from pathlib import Path

______

### Functions

In [120]:
# Create a Folder named Images to save figures in.
IMAGES_PATH = Path.cwd().parent / "Images"
IMAGES_PATH.mkdir(parents=True, exist_ok=True)

def save_fig(fig_id, tight_layout=True, fig_extension="png", resolution=300):
    """
    This functions will save the current figure shown below.
    
    Args:
        fig_id: String Containing the name of the figure.
        tight_layout: Boolean to decide whether you want a tight layout or not.
        fig_extension: String to decide the type of the figure.
        resoultion: Int to decide the resolution of the figure.
        
    Returns:
        None
    """
    path = IMAGES_PATH / f"{fig_id}.{fig_extension}"
    
    if tight_layout:
        plt.tight_layout()
        
    plt.savefig(path, format=fig_extension, dpi=resolution)

In [121]:
def print_unique_values(df, columns):
    """
    Print the unique values for each categorical column and there count in the DataFrame.

    Args:
        df (DataFrame): DataFrame containing categorical columns.
        columns (list): List of column names to loop through.

    Returns:
        None
    """
    for col in columns:
        value_counts = df[col].value_counts().head(5)
        unique_count = len(df[col].unique())
        print(f"Unique values of {col}:\nNo. of Unique values: {unique_count}\n{value_counts}' \n")

In [122]:
def replace_values(df, cols, key_value, type):
    """
    Replace the values of a certain column to a desired value and set the column to the desired type
    
    Args:
        df (DataFrame): DataFrame containing columns you want to replace.
        cols (list): List of column names to loop through.
        key_value (dict): Dictionary contains the original value and new value {'original_value' : 'new_value'}
        type : The desired type of column type = np.int32
        
    Returns:
        None
        
    """
    # loop through each column and replace the key_value pair
    for col in cols:
        df[col] = df[col].replace(key_value).astype(type)

In [123]:
def split_values(df, cols= [], delimiter=','):
    """ 
    Split multiple values in a single string 
    to a list of single strings each represnting a single value. 

    Args:
    df (Dataframe): Dataframe contains desired columns
    delimiter (string): the delimiter to split on ex: ':' 

    Returns:
        None
    """
    #select only object columns
    cols = df[cols].select_dtypes(include='object').columns.tolist()
    
    # loop through object columns and convert to list on the decided delimiter
    for col in cols:
        df[col] = df[col].str.split(delimiter)

In [211]:
def combine_unique_values(df, cols_1, cols_2 , combined_cols):
    """
    Combine related Columns and return unique values
    
    Args:
    df (Dataframe): Dataframe contains desired columns
    cols_1, cols_2 (List): Columns need to be combined 
    combined_cols (list): The new combined Columns
    """
    # change the NAN values to [] to avoid errors
    df[cols_1 + cols_2] = df[cols_1 + cols_2].applymap(lambda x: x if isinstance(x, list) else [])
    
    #loop through the cols
    for i in range(len(cols_1)):
        # create a lambda function to combine each row and return unique values
        combine_lists_unique = lambda row: list(set(row[cols_1[i]] + row[cols_2[i]]))

        # implement the lambda function using apply method and return the new combined column
        df[combined_cols[i]] = df.apply(combine_lists_unique, axis=1)

______

In [124]:
# Load dataset and make a copy
raw_df = pd.read_csv(DATA_PATH)
df = raw_df.copy()

### Check for Duplictes

In [125]:
# Check for duplicates
raw_df.duplicated().value_counts()

False    73268
dtype: int64

- **The Dataset contains no duplicate values.**

### Replace Values and Parse

In [126]:
# Replace Text Values in YearsCode and YearsCodePro to numerical and set type to float
cols = ['YearsCode','YearsCodePro']
REPLACE_YEARS_TEXT = {'Less than 1 year': 0, 'More than 50 years': 51}

replace_values(df, cols, REPLACE_YEARS_TEXT, np.float32)

In [127]:
# Verify your Results
for col in cols: 
    print(col)
    print(df[col].unique().tolist())
    print('--------------------------')
    print()

YearsCode
[nan, 14.0, 20.0, 8.0, 15.0, 3.0, 1.0, 6.0, 37.0, 5.0, 12.0, 22.0, 11.0, 4.0, 7.0, 13.0, 36.0, 2.0, 25.0, 10.0, 40.0, 16.0, 27.0, 24.0, 19.0, 9.0, 17.0, 18.0, 26.0, 51.0, 29.0, 30.0, 32.0, 0.0, 48.0, 45.0, 38.0, 39.0, 28.0, 23.0, 43.0, 21.0, 41.0, 35.0, 50.0, 33.0, 31.0, 34.0, 46.0, 44.0, 42.0, 47.0, 49.0]
--------------------------

YearsCodePro
[nan, 5.0, 17.0, 3.0, 6.0, 30.0, 2.0, 10.0, 15.0, 4.0, 22.0, 20.0, 40.0, 9.0, 14.0, 21.0, 7.0, 18.0, 25.0, 8.0, 12.0, 45.0, 1.0, 19.0, 28.0, 24.0, 11.0, 23.0, 0.0, 32.0, 27.0, 16.0, 44.0, 26.0, 37.0, 46.0, 13.0, 31.0, 39.0, 34.0, 38.0, 35.0, 29.0, 42.0, 36.0, 33.0, 43.0, 41.0, 48.0, 50.0, 51.0, 47.0, 49.0]
--------------------------



### Transform Multiple Values Features

In [128]:
# Split multiple answers in object columns using ';' delimiter
cols = ROLE_COL + CORE_COLS + USEFUL_COLS
split_values(df, cols, delimiter=';')

In [129]:
# Verify your Results
i = df.sample(1).index[0]
print(raw_df['DevType'].iloc[i])
print(df['DevType'].iloc[i])

print(raw_df['LanguageHaveWorkedWith'].iloc[i])
print(df['LanguageHaveWorkedWith'].iloc[i])

Educator;Academic researcher
['Educator', 'Academic researcher']
Fortran;Python
['Fortran', 'Python']


### Combine Related Features

In [209]:
# Create two lists of columns that will be combined.
cols_1 = ['LanguageHaveWorkedWith',
          'DatabaseHaveWorkedWith',
          'PlatformHaveWorkedWith',
          'WebframeHaveWorkedWith',
          'MiscTechHaveWorkedWith',
          'ToolsTechHaveWorkedWith',                               
          'NEWCollabToolsHaveWorkedWith']

cols_2 = ['LanguageWantToWorkWith', 
          'DatabaseWantToWorkWith',  
          'PlatformWantToWorkWith',  
          'WebframeWantToWorkWith',  
          'MiscTechWantToWorkWith',  
          'ToolsTechWantToWorkWith',     
          'NEWCollabToolsWantToWorkWith']

# create a list containing the new names of the combined column.
combined_cols = ['Languages','Databases','Platforms','WebFrameworks','MiscTech','ToolsTech','CollabTools']

combine_unique_values(df,cols_1,cols_2, combined_cols)

In [213]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 73268 entries, 0 to 73267
Data columns (total 86 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   ResponseId                      73268 non-null  int64  
 1   MainBranch                      73268 non-null  object 
 2   Employment                      71709 non-null  object 
 3   RemoteWork                      58958 non-null  object 
 4   CodingActivities                58899 non-null  object 
 5   EdLevel                         71571 non-null  object 
 6   LearnCode                       71580 non-null  object 
 7   LearnCodeOnline                 50685 non-null  object 
 8   LearnCodeCoursesCert            29389 non-null  object 
 9   YearsCode                       71331 non-null  float32
 10  YearsCodePro                    51833 non-null  float32
 11  DevType                         61302 non-null  object 
 12  OrgSize                         

- **Go through the schema of the survey to identify each feature and the question it asks to identify its importance for our business case**
    - **Unuseful features:**
        - `ResponseId`, `SurveyEase`, `SurveyLength`
        - `TrueFalse_1` to `TrueFalse_3`, `Frequency_1` to `Frequency_3`, `Knowledge_1` to `knowledge_7`
        - `Onboarding`, `TimeSearching`, `TimeAnswering`, `ICorPM`, `TBranch`
        - `Trans`, `Sexuality`, `Ethnicity`, `Accessibility`, `MentalHealth`, `Age`, `Gender`, `Blockchain`    
        - `SOComm`, `NEWSOSites`, `SOVisitFreq`, `SOPartFreq`,`SOAccount`, `BuyNewTool`, `PurchaseInfluence`, 
        - `OfficeStackAsyncHaveWorkedWith`, `OfficeStackAsyncWantToWorkWith`, `OfficeStackSyncHaveWorkedWith`, `OfficeStackSyncWantToWorkWith`
        - `VCInteraction`,`VCHostingPersonal use` , `VCHostingProfessional use`
        - `OpSysProfessional use`,  `OpSysPersonal use`,
        
        
    - **Might be useful features:** 
        - `Employment`,`RemoteWork`,
        - `MainBranch`,`CodingActivities`, `ProfessionalTech`,
        - `LearnCode`, `LearnCodeOnline`, `LearnCodeCoursesCert`
        - `WorkExp`, `YearsCode`, `YearsCodePro`, `EdLevel`,
        - `OrgSize`, `Country`,
        - `ConvertedCompYearly`, `Currency`, `CompTotal`, `CompFreq`
        
    - **Core features:** 
        - `DevType`
        - `VersionControlSystem`,   
        - `LanguageHaveWorkedWith`,  `LanguageWantToWorkWith`,  
        - `DatabaseHaveWorkedWith`,  `DatabaseWantToWorkWith`,  
        - `PlatformHaveWorkedWith`,  `PlatformWantToWorkWith`,  
        - `WebframeHaveWorkedWith`,  `WebframeWantToWorkWith`,  
        - `MiscTechHaveWorkedWith`,`MiscTechWantToWorkWith`,  
        - `ToolsTechHaveWorkedWith`,`ToolsTechWantToWorkWith`,                                    
        - `NEWCollabToolsHaveWorkedWith`,  `NEWCollabToolsWantToWorkWith`,  

- **Changes need to be done:**

    - **Get rid of useless features, Coreroles except version control column, Return the only needed features**